In [1]:
import random
import pandas as pd
import numpy as np

import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm
import timm
from data.dataset import train_loader, val_loader, test_loader
from config import *

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
model = timm.create_model("resnet50", pretrained = True, num_classes = 19).to(DEVICE)
le = preprocessing.LabelEncoder()
optimizer = optim.Adam(model.parameters(), lr = LR)
scheduler = CosineAnnealingLR(optimizer, T_max = 32, eta_min = 1e-5)
label_map = {str(i): int(i) for i in range(19)}

In [3]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in (val_loader):
            imgs = imgs.float().to(device)
            labels = [[label_map[label] for label in tpl] for tpl in labels]
            labels = torch.tensor(labels[0], dtype=torch.long).to(DEVICE)

            pred = model(imgs)
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score


def train(model, optimizer, train_loader, val_loader, scheduler):
    best_score = 0
    best_model = None
    criterion = nn.CrossEntropyLoss().to(DEVICE)
    scaler = torch.cuda.amp.GradScaler()
    model.train()
    
    for epoch in (range(EPOCH)):
        batch = 0
        train_loss = []
        with tqdm(train_loader, unit='batch') as tepoch:
            for imgs, labels in tepoch:
                batch += 1
                optimizer.zero_grad()

                tepoch.set_description(f"Epoch {epoch}")
                imgs = imgs.float().to(DEVICE)
                labels = [[label_map[label] for label in tpl] for tpl in labels]
                labels = torch.tensor(labels[0], dtype=torch.long).to(DEVICE)
                
                with torch.cuda.amp.autocast():

                    output = model(imgs).to(DEVICE)
                    loss = criterion(output, labels)
                
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

                train_loss.append(loss.item())

                tepoch.set_postfix(
                    phase="Training",
                    loss=loss.item()
                )

            _val_loss, _val_score = validation(model, criterion, val_loader, DEVICE)
            _train_loss = np.mean(train_loss)
            print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')

            scheduler.step()
                    
            if best_score < _val_score:
                best_score = _val_score
                best_model = model
                print("Best_model saved")
    return best_model


def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm((test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [ ]:
infer_model = train(model, optimizer, train_loader, val_loader, scheduler)

In [ ]:
preds = inference(infer_model, test_loader, DEVICE)

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['label'] = preds
submit.loc[submit['label'] == '0', 'label'] = '가구수정'
submit.loc[submit['label'] == '1', 'label'] = '걸레받이수정'
submit.loc[submit['label'] == '2', 'label'] = '곰팡이'
submit.loc[submit['label'] == '3', 'label'] = '꼬임'
submit.loc[submit['label'] == '4', 'label'] = '녹오염'
submit.loc[submit['label'] == '5', 'label'] = '들뜸'
submit.loc[submit['label'] == '6', 'label'] = '면불량'
submit.loc[submit['label'] == '7', 'label'] = '몰딩수정'
submit.loc[submit['label'] == '8', 'label'] = '반점'
submit.loc[submit['label'] == '9', 'label'] = '석고수정'
submit.loc[submit['label'] == '10', 'label'] = '오염'
submit.loc[submit['label'] == '11', 'label'] = '오타공'
submit.loc[submit['label'] == '12', 'label'] = '울음'
submit.loc[submit['label'] == '13', 'label'] = '이음부불량'
submit.loc[submit['label'] == '14', 'label'] = '창틀,문틀수정'
submit.loc[submit['label'] == '15', 'label'] = '터짐'
submit.loc[submit['label'] == '16', 'label'] = '틈새과다'
submit.loc[submit['label'] == '17', 'label'] = '피스'
submit.loc[submit['label'] == '18', 'label'] = '훼손'

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False, encoding='utf-8')

In [32]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from pathlib import Path
import cv2

class TestDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = Path(self.img_path_list[index])
        image = cv2.imread(str(img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)
    

test_transform = A.Compose([
                            A.Resize(360,360),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test = pd.read_csv('C:/Users/lee97/Documents/GitHub/DeepLearning/dacon/src/data/test.csv')
test_dataset = TestDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)
test_labels = le.transform(test['label'])


KeyError: 'label'

In [ ]:
preds = inference(infer_model, test_loader, DEVICE)

In [33]:
all_img_list = glob.glob('./train/*/*')
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[1])
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=41)
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])


ValueError: With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.